## **Imports**

In [14]:
# db
import aiosqlite

# web
import aiohttp
import asyncio

# analysis 
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

# system
import os
import sys
import shutil

## **Database**

In [15]:
DATABASE_NAME = 'sites.db'

In [16]:
async def fill_database(name=DATABASE_NAME, rewrite=True):
    if rewrite and os.path.isfile(name):
        os.remove(name)
    async with aiosqlite.connect(DATABASE_NAME) as db: 
        await db.execute('''
        CREATE TABLE pages (
          id INTEGER PRIMARY KEY AUTOINCREMENT,
          url TEXT,
          page_name TEXT,
          
          pure_html TEXT,
          plain_text TEXT
        )
        ''')
        
        await db.execute('''
        CREATE TABLE children (
          id INTEGER PRIMARY KEY AUTOINCREMENT,
          root_id INTEGER,
          child_id INTEGER,
          FOREIGN KEY (root_id)  REFERENCES pages(id),
          FOREIGN KEY (child_id)  REFERENCES pages(id)
        )
        ''')
        
        await db.execute('''
        CREATE TABLE sites (
          id INTEGER PRIMARY KEY AUTOINCREMENT,
          url TEXT,
          site_name TEXT,
          root_id INTEGER,
          FOREIGN KEY (root_id)  REFERENCES pages(id)
        )
        ''')
        
        for page in pages:
            await db.execute('INSERT INTO pages VALUES(?, ?, ?, ?, ?)', page)
        for site in sites:
            await db.execute('INSERT INTO sites VALUES(?, ?, ?, ?)', site)
        for rel in children:
            await db.execute('INSERT INTO childern VALUES(?, ?, ?)', rel)
            
        await db.commit()
        

## **Get site data**

In [17]:
pages = []
children = []
sites = []

In [18]:
headers = {
    'Accept-Language': 'ru,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
}

In [19]:
def make_links_absolute(html, url):
    absolutize = lambda m: ' src="' + urljoin(url, m.group(1)) + '"'
    html = re.sub(r' src="([^"]+)"', absolutize, html)
    absolutize2 = lambda m: ' href="' + urljoin(url, m.group(1)) + '"'
    html = re.sub(r' href="([^"]+)"', absolutize2, html)
    return html

In [20]:
async def get_plain_text(soup):
    for script in soup(["script", "style"]):
      script.extract()

    text = soup.get_text()
    
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    return '\n'.join(chunk for chunk in chunks if chunk)

In [21]:
async def get_links(soup):
    for script in soup(["script", "style"]):
      script.extract()

    text = soup.get_text()
    
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    return '\n'.join(chunk for chunk in chunks if chunk)

In [22]:
async def get_page(session: aiohttp.ClientSession, url: str, root = 0):
    try:
        await asyncio.sleep(0.1)
        async with session.get(url, headers=headers) as response:
            await asyncio.sleep(0.01)
            text = await response.text(errors='replace')
            text = make_links_absolute(text, url)
            text = text[:text.rfind('<!-- copyright (t2) -->')]
            
            soup = BeautifulSoup(text, features="html.parser")
            plain_text = await get_plain_text(soup)
            title = soup.title.string if soup.title else ""
            
            index = len(pages) + 1
            pages.append((index, url, str(title), text, plain_text))
            
            if root > 0:
                ch_i = len(children) + 1
                children.append((ch_i, root, index))
            
            site = url[url.find('://') + 3:]
            site = site[:site.find('/')]
            tasks = []
            for link in soup.findAll('a'):
                link = link.get('href')
                if link and site in link and link.startswith('http'):
                    task = asyncio.create_task(get_page(session, link))
                    tasks.append(task)
                    
            await asyncio.gather(*tasks)
            
            return index
    except aiohttp.ClientConnectorError:
        print(url)
    except Exception as e:
        print(e)

In [23]:
async def get_site(url: str, sem: asyncio.Semaphore):
    async with aiohttp.ClientSession() as session:
        async with sem:
            root = await get_page(session, url)
            index = len(sites) + 1
            print(index)
            sites.append((index, url, pages[root - 1][2], root))

In [24]:
async def gather_sites(sites):
    sem = asyncio.Semaphore(5)
    tasks = []
    
    for site_link in sites:
        task = asyncio.create_task(get_site(site_link, sem))
        tasks.append(task)
    
    await asyncio.gather(*tasks)
    
    await fill_database()

In [25]:
async def main():
    sites = [i.strip() for i in open('Narod.txt')]
    await gather_sites(sites)

In [26]:
await main()

Response payload is not completed
[Errno 54] Connection reset by peer
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected
Server disconnected




































































































































TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'